In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('../data/raw/gmail_emails.csv')
df.head()

,sender,receivers,message
0,Gmail Team <mail-noreply@google.com>,Karthik Gnana Ezhil S <ep23btech11016@iith.ac.in>,Get the official Gmail app\n\nGet the official...
1,Gmail Team <mail-noreply@google.com>,Karthik Gnana Ezhil S <ep23btech11016@iith.ac.in>,Tips for using your new inbox\n\nTips for usin...
2,SKGEzhil <skgezhil2005@gmail.com>,"""ep23btech11016@iith.ac.in"" <ep23btech11016@ii...",\n\n\r\n
3,International Collaboration IR <ic.ir@iith.ac.in>,students@iith.ac.in,Call for applications for the Swiss Government...
4,Green Office IIT Hyderabad <office.green@iith....,"Students <students@iith.ac.in>, ""Faculty, IIT ...","Plantation drive on 05-08-2023\n\nDear All,\r\..."


In [4]:
df['message'][0]

'Get the official Gmail app\n\nGet the official Gmail app\r\n[image: Gmail]\r\n\r\nGet the official Gmail app\r\nThe best features of IIT Hyderabad Mail are only available on your phone\r\nand tablet with the official Gmail app. Download the app or go to gmail.com\r\non your computer or mobile device to get started.\r\n\r\n[image: Get it on Google Play]\r\n<https://play.google.com/store/apps/details?id=com.google.android.gm&hl=en>\r\n[image:\r\nApple App Store]\r\n<https://apps.apple.com/app/gmail-email-by-google/id422689480?l=en>\r\n\r\nHappy emailing,\r\nThe Gmail Team\r\n[image: Google Cloud]\r\n\r\n© 2023 Google LLC 1600 Amphitheatre Parkway, Mountain View, CA 94043\r\n\r\n*You’re receiving this mandatory email service announcement to update you\r\nabout important changes to your Google Cloud product or account.*\r\n'

In [3]:
from sentence_transformers import SentenceTransformer, util

# 2) Load SBERT and encode all emails
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(df['message'].tolist(), convert_to_tensor=True)


/opt/anaconda3/envs/email-classifier-env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [42]:
import torch

# `embeddings` is your tensor of shape [14000, D]
torch.save(embeddings, '../data/processed/email_embeddings.pt')


In [93]:
df['emb'] = embeddings.tolist()  # convert tensor to list for DataFrame

In [94]:
import numpy as np

raw_prototypes = {
    'meeting':     ["Team sync-up meeting", "Project planning meeting"],
    'announcement':["Official announcement", "Please note the announcement"],
    'internship':  ["Summer internship opportunity", "Apply for internship", "Internship application deadline", "Internship program details", "Registration deadline for the internship", "Exchange program opportunity", "exchange program"],
    'club':        ["Student club event", "Join our club", "Club orientations", "Sunshine meeting invites", "shuffle conducting events", "Vibes conducting events", "milan events", "Elan and nvision", "Diesta events", "Behind the lens events"],
    'talks':       ["Guest speaker talk", "Research seminar talks", "Invitation for seminar", "viva voice", "Upcoming talks", "Research talks", "Sessions on academics", "Upcoming events", "Event registration", "Invitation to event"],
    'maintenance':  ["scheduled maintenance", "Maintenance window", "Internet connection down", "Shutdown notice"],
    'academics':   ["Academic calendar", "Course registration", "Exam schedule", "Assignment deadlines", "Assignment submission"],
}
prototypes = {
    label: np.mean(
        model.encode(texts, convert_to_tensor=False),  # now returns a list of lists
        axis=0
    )
    for label, texts in raw_prototypes.items()
}



In [95]:
# 4) Compute cosine‐similarities of each email to each prototype
sims = {
    label: util.cos_sim(proto, embeddings)[0]
    for label, proto in prototypes.items()
}

threshold = 0.4
def pick_label(i):
    # build a dict of {label: score}
    scores = {lbl: sims[lbl][i].item() for lbl in sims}
    best_label, best_score = max(scores.items(), key=lambda kv: kv[1])
    return best_label if best_score > threshold else 'other'

df['label'] = [pick_label(i) for i in range(len(df))]

# print(df)
df

,sender,receivers,message,label,emb
0,Gmail Team <mail-noreply@google.com>,Karthik Gnana Ezhil S <ep23btech11016@iith.ac.in>,Get the official Gmail app\n\nGet the official...,other,"[-0.09933263063430786, 0.001328121405094862, 0..."
1,Gmail Team <mail-noreply@google.com>,Karthik Gnana Ezhil S <ep23btech11016@iith.ac.in>,Tips for using your new inbox\n\nTips for usin...,other,"[-0.050356633961200714, 0.06010790541768074, -..."
2,SKGEzhil <skgezhil2005@gmail.com>,"""ep23btech11016@iith.ac.in"" <ep23btech11016@ii...",\n\n\r\n,other,"[-0.11883833259344101, 0.0482986718416214, -0...."
3,International Collaboration IR <ic.ir@iith.ac.in>,students@iith.ac.in,Call for applications for the Swiss Government...,other,"[-0.09319693595170975, 0.01009377185255289, -0..."
4,Green Office IIT Hyderabad <office.green@iith....,"Students <students@iith.ac.in>, ""Faculty, IIT ...","Plantation drive on 05-08-2023\n\nDear All,\r\...",other,"[-0.04429999738931656, -0.015538915991783142, ..."
...,...,...,...,...,...
18946,Shiv Govind Singh <sgsingh@ee.iith.ac.in>,"""EE, Seminar"" <seminar@ee.iith.ac.in>, ""Ph.D. ...","PhD Vivo-Voce (online) of Ghosh Tanmoya , 2.30...",other,"[-0.10579375177621841, -0.015991903841495514, ..."
18947,Japandesk IR <japandesk.ir@iith.ac.in>,Students <students@iith.ac.in>,Re: BTech only : The University of Tokyo Sakur...,academics,"[-0.11099039763212204, -0.015269726514816284, ..."
18948,TechGig Webinar <expertspeak@techgig.com>,ep23btech11016@iith.ac.in,How UiPath Autopilot is Redefining Developer P...,other,"[-0.018514640629291534, -0.06346655637025833, ..."
18949,"""'Cultural Secretary' via Students, IIT Hydera...",Students <students@iith.ac.in>,Re: Join the Shuffle Club’s Online Dance Reel ...,club,"[-0.10851454734802246, -0.05394021049141884, 0..."
